In [24]:
input_file1 = "QM9_129440_MLtraining"
input_file2 = "QM9_49762_MLtraining"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pickle
import os
df1 = pd.read_csv(f'{input_file1}.csv')
df2 = pd.read_csv(f'{input_file2}.csv')
print(df2.info())
print(df2.describe())

print(df2.isnull().sum().sort_values(ascending=False))
df2 = df2.dropna()
df2 = df2.replace([np.inf, -np.inf], np.nan).dropna()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Phân phối target variables
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for i, target in enumerate(['bps_pred', 'mps_pred', 'fps_pred']):
    sns.histplot(df2[target], kde=True, ax=axes[i])
    axes[i].set_title(f'Phân phối của {target}')
plt.show()

# Phân phối các features numeric
numeric_cols = df2.select_dtypes(include=['float64', 'int64']).columns
df2[numeric_cols].hist(bins=30, figsize=(20, 15))
plt.tight_layout()
plt.show()

In [ ]:
# Ma trận tương quan
corr_matrix = df2.select_dtypes(include=['float64', 'int64']).corr()
plt.figure(figsize=(20, 15))
sns.heatmap(corr_matrix[(corr_matrix > 0.4) | (corr_matrix < -0.4)], 
            annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix (|corr| > 0.5)')
plt.show()

# Tương quan với target
targets = ['bps_pred', 'mps_pred', 'fps_pred']
for target in targets:
    corr_with_target = df2.select_dtypes(include=['float64', 'int64']).corr()[target].sort_values(ascending=False)
    print(f"\nTương quan với {target}:")
    print(corr_with_target[abs(corr_with_target) > 0.3])

In [ ]:
# frature importance từ mô hình
from sklearn.ensemble import RandomForestRegressor

# Tính feature importance cho từng target
for target in ['bps_pred', 'mps_pred', 'fps_pred']:
    X = df2.drop(columns=['bps_pred', 'mps_pred', 'fps_pred', 'canonical_smiles'])
    y = df2[target]
    
    model = RandomForestRegressor(n_estimators=100, random_state=7)
    model.fit(X, y)
    
    importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nFeature importance cho {target}:")
    print(importance.head(20))
    
    # Visualize top 20 features
    plt.figure(figsize=(10, 6))
    sns.barplot(x='importance', y='feature', data=importance.head(30))
    plt.title(f'Top 20 Features for {target}')
    plt.show()

In [ ]:
# chọn feaatures bằng thống kê
from sklearn.feature_selection import SelectKBest, mutual_info_regression

for target in ['bps_pred', 'mps_pred', 'fps_pred']:
    X = df2.drop(columns=['bps_pred', 'mps_pred', 'fps_pred', 'canonical_smiles'])
    y = df2[target]
    
    # Chọn top k features dựa trên mutual information
    selector = SelectKBest(score_func=mutual_info_regression, k=20)
    selector.fit(X, y)
    
    selected_features = X.columns[selector.get_support()]
    scores = selector.scores_[selector.get_support()]
    
    print(f"\nTop features cho {target} (Mutual Info):")
    print(pd.DataFrame({'feature': selected_features, 'score': scores})
          .sort_values('score', ascending=False))

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# PCA để giảm chiều và chọn features quan trọng
for target in ['bps_pred', 'mps_pred', 'fps_pred']:
    X = df2.drop(columns=['bps_pred', 'mps_pred', 'fps_pred', 'canonical_smiles'])
    y = df2[target]
    
    # Chuẩn hóa dữ liệu trước khi thực hiện PCA
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Thực hiện PCA
    pca = PCA(n_components=0.9999)  # Giữ lại các thành phần giải thích 95% phương sai
    pca_result = pca.fit_transform(X_scaled)
    
    # In ra số lượng thành phần được giữ lại
    print(f"\nPCA cho {target}:")
    print(f"Số lượng thành phần chính được giữ lại: {pca.n_components_}")
    print(f"Tổng phương sai giải thích được: {sum(pca.explained_variance_ratio_):.4f}")
    
    # Hiển thị tỷ lệ phương sai được giải thích bởi mỗi thành phần
    plt.figure(figsize=(10, 6))
    plt.bar(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_)
    plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), 
             np.cumsum(pca.explained_variance_ratio_), 'r-')
    plt.xlabel('Thành phần chính')
    plt.ylabel('Tỷ lệ phương sai giải thích')
    plt.title(f'Scree Plot cho {target}')
    plt.show()
    
    # Xác định các features quan trọng nhất thông qua PCA loadings
    loadings = pd.DataFrame(
        pca.components_.T,
        columns=[f'PC{i+1}' for i in range(pca.n_components_)],
        index=X.columns
    )
    
    # Lấy top features có ảnh hưởng lớn nhất đến các principal components
    abs_loadings = abs(loadings).sum(axis=1).sort_values(ascending=False)
    print("\nTop 20 features quan trọng nhất theo PCA:")
    print(abs_loadings.head(20))